# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Prompt for GPT
prompt_raw = "I would like to summarise in maximum 300 characters and in English what the following sentences in Portuguese talk about. \
Do not use aspects mentioned only in one sentence as part of the summary: TWEETS"

# 2015 negative

## January

In [4]:
# Get monthly data
df_2015_01_neg = pd.read_csv('../data/local/df_neg_2015-01.csv').reset_index()
df_2015_01_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   543 non-null    int64  
 1   LocationCode            543 non-null    object 
 2   LocationName            543 non-null    object 
 3   StatesCode              543 non-null    object 
 4   ...4                    543 non-null    int64  
 5   id                      543 non-null    float64
 6   text                    543 non-null    object 
 7   label                   543 non-null    object 
 8   in_reply_to_status_id   66 non-null     float64
 9   in_reply_to_user_id     69 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       107 non-null    float64
 13  retweeted_status_id     107 non-null    float64
 14  created_at              543 non-null    ob

### Representative sample based on week and location

In [5]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_01_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_01_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w01,Acre,1,0.001842
1,2015-w01,Amazonas,5,0.009208
2,2015-w01,Bahia,5,0.009208
3,2015-w01,Ceara,1,0.001842
4,2015-w01,Espirito Santo,1,0.001842
...,...,...,...,...
76,2015-w05,Paraiba,4,0.007366
77,2015-w05,Parana,4,0.007366
78,2015-w05,Rio Grande Do Sul,6,0.011050
79,2015-w05,Rondonia,1,0.001842


In [6]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w01,Acre,1,0.001842,4
1,2015-w01,Amazonas,5,0.009208,22
2,2015-w01,Bahia,5,0.009208,22
3,2015-w01,Ceara,1,0.001842,4
4,2015-w01,Espirito Santo,1,0.001842,4
...,...,...,...,...,...
76,2015-w05,Paraiba,4,0.007366,18
77,2015-w05,Parana,4,0.007366,18
78,2015-w05,Rio Grande Do Sul,6,0.011050,27
79,2015-w05,Rondonia,1,0.001842,4


In [7]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_01_neg_sample_size = pd.merge(df_2015_01_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [8]:
# Step 6: Perform the stratified sampling
#df_2015_01_neg_sample = df_2015_01_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_01_neg_sample = df_2015_01_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_01_neg_sample)} rows.")


Sampled 543 rows.


In [9]:
text_gpt_2015_01_neg = '\n'.join(df_2015_01_neg_sample['text'].astype(str))

### Prompts and loops

In [10]:
prompt_1 = prompt_raw.replace('TWEETS', text_gpt_2015_01_neg)

In [11]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_1}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [12]:
response.choices[0].message.content

'The text reflects a widespread frustration and disdain for vaccinations. Many express discomfort, pain, or adverse reactions after receiving vaccines, while also highlighting a growing anti-vaccine movement. Concerns about vaccine safety, the rising costs of childhood vaccinations, and the consequences of low vaccination rates—such as disease outbreaks—are prevalent themes.'

In [13]:
summary = []
dataset = []

In [14]:
dataset.append("2015_01_negitive")

In [15]:
summary.append(response.choices[0].message.content)

In [16]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1 non-null      object
 1   summary  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [17]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...


In [18]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd41xcYwq9EdPlfATpw691RJQjxM at 0x24359af1270> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text reflects a widespread frustration and disdain for vaccinations. Many express discomfort, pain, or adverse reactions after receiving vaccines, while also highlighting a growing anti-vaccine movement. Concerns about vaccine safety, the rising costs of childhood vaccinations, and the consequences of low vaccination rates\u2014such as disease outbreaks\u2014are prevalent themes.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863365,
  "id": "chatcmpl-AUd41xcYwq9EdPlfATpw691RJQjxM",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "a

## February

In [19]:
# Get monthly data
df_2015_02_neg = pd.read_csv('../data/local/df_neg_2015-02.csv').reset_index()
df_2015_02_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   474 non-null    int64  
 1   LocationCode            474 non-null    object 
 2   LocationName            474 non-null    object 
 3   StatesCode              474 non-null    object 
 4   ...4                    474 non-null    int64  
 5   id                      474 non-null    float64
 6   text                    474 non-null    object 
 7   label                   474 non-null    object 
 8   in_reply_to_status_id   62 non-null     float64
 9   in_reply_to_user_id     64 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       99 non-null     float64
 13  retweeted_status_id     99 non-null     float64
 14  created_at              474 non-null    ob

### Representative sample based on week and location

In [20]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_02_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_02_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w05,Amazonas,1,0.002110
1,2015-w05,Bahia,3,0.006329
2,2015-w05,Ceara,2,0.004219
3,2015-w05,Distrito Federal,3,0.006329
4,2015-w05,Espirito Santo,1,0.002110
...,...,...,...,...
76,2015-w09,Minas Gerais,7,0.014768
77,2015-w09,Para,1,0.002110
78,2015-w09,Parana,11,0.023207
79,2015-w09,Rio Grande Do Sul,5,0.010549


In [21]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w05,Amazonas,1,0.002110,5
1,2015-w05,Bahia,3,0.006329,15
2,2015-w05,Ceara,2,0.004219,10
3,2015-w05,Distrito Federal,3,0.006329,15
4,2015-w05,Espirito Santo,1,0.002110,5
...,...,...,...,...,...
76,2015-w09,Minas Gerais,7,0.014768,36
77,2015-w09,Para,1,0.002110,5
78,2015-w09,Parana,11,0.023207,56
79,2015-w09,Rio Grande Do Sul,5,0.010549,25


In [22]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_02_neg_sample_size = pd.merge(df_2015_02_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [23]:
# Step 6: Perform the stratified sampling
#df_2015_02_neg_sample = df_2015_02_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_02_neg_sample = df_2015_02_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_02_neg_sample)} rows.")


Sampled 474 rows.


In [24]:
text_gpt_2015_02_neg = '\n'.join(df_2015_02_neg_sample['text'].astype(str))

### Prompts and loops

In [25]:
prompt_2 = prompt_raw.replace('TWEETS', text_gpt_2015_02_neg)

In [26]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_2}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [27]:
response.choices[0].message.content

'The selected texts express a mix of opinions and experiences regarding vaccinations. They highlight frustration with the pain associated with vaccines, concerns about vaccine efficacy and safety, and the rise of anti-vaccine sentiments. Additionally, there are criticisms of those who refuse vaccinations for their children, linking it to public health risks, while some express a desire for vaccines to prevent various issues, including social behaviors and diseases.'

In [28]:
#summary = []
#dataset = []

In [29]:
dataset.append("2015_02_negitive")

In [30]:
summary.append(response.choices[0].message.content)

In [31]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  2 non-null      object
 1   summary  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [32]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...


In [33]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd44NUJkSB163lWo46Hafn00VI77 at 0x24359b1e860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The selected texts express a mix of opinions and experiences regarding vaccinations. They highlight frustration with the pain associated with vaccines, concerns about vaccine efficacy and safety, and the rise of anti-vaccine sentiments. Additionally, there are criticisms of those who refuse vaccinations for their children, linking it to public health risks, while some express a desire for vaccines to prevent various issues, including social behaviors and diseases.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863368,
  "id": "chatcmpl-AUd44NUJkSB163lWo46Hafn00VI77",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 78,


## March

In [34]:
# Get monthly data
df_2015_03_neg = pd.read_csv('../data/local/df_neg_2015-03.csv').reset_index()
df_2015_03_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   728 non-null    int64  
 1   LocationCode            728 non-null    object 
 2   LocationName            728 non-null    object 
 3   StatesCode              728 non-null    object 
 4   ...4                    728 non-null    int64  
 5   id                      728 non-null    float64
 6   text                    728 non-null    object 
 7   label                   728 non-null    object 
 8   in_reply_to_status_id   77 non-null     float64
 9   in_reply_to_user_id     82 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       98 non-null     float64
 13  retweeted_status_id     98 non-null     float64
 14  created_at              728 non-null    ob

### Representative sample based on week and location

In [35]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_03_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_03_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w09,Acre,1,0.001374
1,2015-w09,Amazonas,2,0.002747
2,2015-w09,Bahia,6,0.008242
3,2015-w09,Ceara,3,0.004121
4,2015-w09,Distrito Federal,2,0.002747
...,...,...,...,...
89,2015-w13,Rio Grande Do Sul,10,0.013736
90,2015-w13,Roraima,1,0.001374
91,2015-w13,Santa Catarina,1,0.001374
92,2015-w13,Sao Paulo,27,0.037088


In [36]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w09,Acre,1,0.001374,3
1,2015-w09,Amazonas,2,0.002747,6
2,2015-w09,Bahia,6,0.008242,20
3,2015-w09,Ceara,3,0.004121,10
4,2015-w09,Distrito Federal,2,0.002747,6
...,...,...,...,...,...
89,2015-w13,Rio Grande Do Sul,10,0.013736,33
90,2015-w13,Roraima,1,0.001374,3
91,2015-w13,Santa Catarina,1,0.001374,3
92,2015-w13,Sao Paulo,27,0.037088,90


In [37]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_03_neg_sample_size = pd.merge(df_2015_03_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [38]:
# Step 6: Perform the stratified sampling
#df_2015_03_neg_sample = df_2015_03_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_03_neg_sample = df_2015_03_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_03_neg_sample)} rows.")


Sampled 728 rows.


In [39]:
text_gpt_2015_03_neg = '\n'.join(df_2015_03_neg_sample['text'].astype(str))

### Prompts and loops

In [40]:
prompt_3 = prompt_raw.replace('TWEETS', text_gpt_2015_03_neg)

In [41]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_3}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [42]:
response.choices[0].message.content

'The discussion revolves around various concerns and experiences related to vaccinations. It highlights side effects, public sentiment against HPV vaccines, the lack of vaccines (especially BCG and anti-tuberculosis) in Brazil, and tragic incidents linked to vaccination campaigns, including attacks on vaccination efforts. Reactions range from fear and skepticism to frustration over vaccine shortages and government negligence.'

In [43]:
#summary = []
#dataset = []

In [44]:
dataset.append("2015_03_negitive")

In [45]:
summary.append(response.choices[0].message.content)

In [46]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  3 non-null      object
 1   summary  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [47]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...


In [48]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd46JO1UcsUZgtjGmjuB5bHEhtkl at 0x24359ce2d10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The discussion revolves around various concerns and experiences related to vaccinations. It highlights side effects, public sentiment against HPV vaccines, the lack of vaccines (especially BCG and anti-tuberculosis) in Brazil, and tragic incidents linked to vaccination campaigns, including attacks on vaccination efforts. Reactions range from fear and skepticism to frustration over vaccine shortages and government negligence.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863370,
  "id": "chatcmpl-AUd46JO1UcsUZgtjGmjuB5bHEhtkl",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 70,
    "completion_tokens_details": {
     

## April

In [49]:
# Get monthly data
df_2015_04_neg = pd.read_csv('../data/local/df_neg_2015-04.csv').reset_index()
df_2015_04_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   664 non-null    int64  
 1   LocationCode            664 non-null    object 
 2   LocationName            664 non-null    object 
 3   StatesCode              664 non-null    object 
 4   ...4                    664 non-null    int64  
 5   id                      664 non-null    float64
 6   text                    664 non-null    object 
 7   label                   664 non-null    object 
 8   in_reply_to_status_id   67 non-null     float64
 9   in_reply_to_user_id     72 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       107 non-null    float64
 13  retweeted_status_id     107 non-null    float64
 14  created_at              664 non-null    ob

### Representative sample based on week and location

In [50]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_04_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_04_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w13,Acre,2,0.003012
1,2015-w13,Bahia,1,0.001506
2,2015-w13,Ceara,3,0.004518
3,2015-w13,Distrito Federal,2,0.003012
4,2015-w13,Maranhao,3,0.004518
...,...,...,...,...
81,2015-w18,Minas Gerais,8,0.012048
82,2015-w18,Paraiba,4,0.006024
83,2015-w18,Parana,3,0.004518
84,2015-w18,Rio Grande Do Sul,11,0.016566


In [51]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w13,Acre,2,0.003012,7
1,2015-w13,Bahia,1,0.001506,3
2,2015-w13,Ceara,3,0.004518,11
3,2015-w13,Distrito Federal,2,0.003012,7
4,2015-w13,Maranhao,3,0.004518,11
...,...,...,...,...,...
81,2015-w18,Minas Gerais,8,0.012048,29
82,2015-w18,Paraiba,4,0.006024,14
83,2015-w18,Parana,3,0.004518,11
84,2015-w18,Rio Grande Do Sul,11,0.016566,40


In [52]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_04_neg_sample_size = pd.merge(df_2015_04_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [54]:
# Step 6: Perform the stratified sampling
#df_2015_04_neg_sample = df_2015_04_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_04_neg_sample = df_2015_04_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_04_neg_sample)} rows.")


Sampled 664 rows.


In [55]:
text_gpt_2015_04_neg = '\n'.join(df_2015_04_neg_sample['text'].astype(str))

### Prompts and loops

In [56]:
prompt_4 = prompt_raw.replace('TWEETS', text_gpt_2015_04_neg)

In [57]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_4}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [58]:
response.choices[0].message.content

'The sentences express various opinions and experiences regarding vaccinations, highlighting discomfort and fear associated with them. There are mentions of specific vaccines like HPV and tetanus, concerns about side effects, and instances of people avoiding vaccination. Some express distrust in vaccines, linking them to government scandals or personal anecdotes, while others note incidents of vaccine shortages and criminal activities at vaccination clinics.'

In [59]:
#summary = []
#dataset = []

In [60]:
dataset.append("2015_04_negitive")

In [61]:
summary.append(response.choices[0].message.content)

In [62]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  4 non-null      object
 1   summary  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [63]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...


In [64]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd4gYniLaiRrJwaReMMKmhMEKaSL at 0x2435e3b4f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express various opinions and experiences regarding vaccinations, highlighting discomfort and fear associated with them. There are mentions of specific vaccines like HPV and tetanus, concerns about side effects, and instances of people avoiding vaccination. Some express distrust in vaccines, linking them to government scandals or personal anecdotes, while others note incidents of vaccine shortages and criminal activities at vaccination clinics.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863406,
  "id": "chatcmpl-AUd4gYniLaiRrJwaReMMKmhMEKaSL",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 71,
    "co

## May

In [65]:
# Get monthly data
df_2015_05_neg = pd.read_csv('../data/local/df_neg_2015-05.csv').reset_index()
df_2015_05_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2245 non-null   int64  
 1   LocationCode            2245 non-null   object 
 2   LocationName            2245 non-null   object 
 3   StatesCode              2245 non-null   object 
 4   ...4                    2245 non-null   int64  
 5   id                      2245 non-null   float64
 6   text                    2245 non-null   object 
 7   label                   2245 non-null   object 
 8   in_reply_to_status_id   174 non-null    float64
 9   in_reply_to_user_id     191 non-null    float64
 10  quoted_user_id          2 non-null      float64
 11  quoted_status_id        2 non-null      float64
 12  retweeted_user_id       197 non-null    float64
 13  retweeted_status_id     197 non-null    float64
 14  created_at              2245 non-null   

### Representative sample based on week and location

In [66]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_05_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_05_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w18,Alagoas,7,0.003118
1,2015-w18,Amapa,1,0.000445
2,2015-w18,Amazonas,5,0.002227
3,2015-w18,Bahia,15,0.006682
4,2015-w18,Ceara,6,0.002673
...,...,...,...,...
109,2015-w22,Rio De Janeiro,2,0.000891
110,2015-w22,Rio Grande Do Sul,31,0.013808
111,2015-w22,Santa Catarina,7,0.003118
112,2015-w22,Sao Paulo,35,0.015590


In [67]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w18,Alagoas,7,0.003118,7
1,2015-w18,Amapa,1,0.000445,1
2,2015-w18,Amazonas,5,0.002227,5
3,2015-w18,Bahia,15,0.006682,16
4,2015-w18,Ceara,6,0.002673,6
...,...,...,...,...,...
109,2015-w22,Rio De Janeiro,2,0.000891,2
110,2015-w22,Rio Grande Do Sul,31,0.013808,33
111,2015-w22,Santa Catarina,7,0.003118,7
112,2015-w22,Sao Paulo,35,0.015590,38


In [68]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_05_neg_sample_size = pd.merge(df_2015_05_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [70]:
# Step 6: Perform the stratified sampling
#df_2015_05_neg_sample = df_2015_05_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_05_neg_sample = df_2015_05_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_05_neg_sample)} rows.")


Sampled 2245 rows.


In [71]:
text_gpt_2015_05_neg = '\n'.join(df_2015_05_neg_sample['text'].astype(str))

### Prompts and loops

In [72]:
prompt_5 = prompt_raw.replace('TWEETS', text_gpt_2015_05_neg)

In [73]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_5}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [74]:
response.choices[0].message.content

"The text discusses various people's experiences and sentiments regarding vaccinations, particularly the flu vaccine. Many express discomfort and pain after receiving vaccines, while some question their effectiveness or express fear of needles. There's also mention of vaccination campaigns, shortages, and controversies surrounding vaccines, including a focus on HPV vaccines. Overall, the sentiment reflects a mix of annoyance, fear, and skepticism towards vaccination practices and their side effects."

In [75]:
#summary = []
#dataset = []

In [76]:
dataset.append("2015_05_negitive")

In [77]:
summary.append(response.choices[0].message.content)

In [78]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  5 non-null      object
 1   summary  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [79]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...


In [80]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5CSBvk6z80AhpLlTuvCusiePXD at 0x2435e5bad10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various people's experiences and sentiments regarding vaccinations, particularly the flu vaccine. Many express discomfort and pain after receiving vaccines, while some question their effectiveness or express fear of needles. There's also mention of vaccination campaigns, shortages, and controversies surrounding vaccines, including a focus on HPV vaccines. Overall, the sentiment reflects a mix of annoyance, fear, and skepticism towards vaccination practices and their side effects.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863438,
  "id": "chatcmpl-AUd5CSBvk6z80AhpLlTuvCusiePXD",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usag

## June

In [81]:
# Get monthly data
df_2015_06_neg = pd.read_csv('../data/local/df_neg_2015-06.csv').reset_index()
df_2015_06_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189 entries, 0 to 1188
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1189 non-null   int64  
 1   LocationCode            1189 non-null   object 
 2   LocationName            1189 non-null   object 
 3   StatesCode              1189 non-null   object 
 4   ...4                    1189 non-null   int64  
 5   id                      1189 non-null   float64
 6   text                    1189 non-null   object 
 7   label                   1189 non-null   object 
 8   in_reply_to_status_id   107 non-null    float64
 9   in_reply_to_user_id     110 non-null    float64
 10  quoted_user_id          6 non-null      float64
 11  quoted_status_id        6 non-null      float64
 12  retweeted_user_id       123 non-null    float64
 13  retweeted_status_id     123 non-null    float64
 14  created_at              1189 non-null   

### Representative sample based on week and location

In [82]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_06_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_06_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w22,Acre,2,0.001682
1,2015-w22,Alagoas,1,0.000841
2,2015-w22,Amapa,2,0.001682
3,2015-w22,Amazonas,2,0.001682
4,2015-w22,Bahia,8,0.006728
...,...,...,...,...
98,2015-w26,Pernambuco,6,0.005046
99,2015-w26,Rio Grande Do Sul,34,0.028595
100,2015-w26,Santa Catarina,8,0.006728
101,2015-w26,Sao Paulo,41,0.034483


In [83]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w22,Acre,2,0.001682,4
1,2015-w22,Alagoas,1,0.000841,2
2,2015-w22,Amapa,2,0.001682,4
3,2015-w22,Amazonas,2,0.001682,4
4,2015-w22,Bahia,8,0.006728,16
...,...,...,...,...,...
98,2015-w26,Pernambuco,6,0.005046,12
99,2015-w26,Rio Grande Do Sul,34,0.028595,70
100,2015-w26,Santa Catarina,8,0.006728,16
101,2015-w26,Sao Paulo,41,0.034483,84


In [84]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_06_neg_sample_size = pd.merge(df_2015_06_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [85]:
# Step 6: Perform the stratified sampling
#df_2015_06_neg_sample = df_2015_06_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_06_neg_sample = df_2015_06_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_06_neg_sample)} rows.")


Sampled 1189 rows.


In [86]:
text_gpt_2015_06_neg = '\n'.join(df_2015_06_neg_sample['text'].astype(str))

### Prompts and loops

In [87]:
prompt_6 = prompt_raw.replace('TWEETS', text_gpt_2015_06_neg)

In [88]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_6}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [89]:
response.choices[0].message.content

'The text discusses various aspects of vaccination, particularly against the flu and other diseases. It highlights issues like low vaccination rates, painful experiences associated with getting vaccinated, and skepticism about vaccine efficacy, as well as concerns about potential health risks. Additionally, it mentions a resurgence of diseases like diphtheria due to anti-vaccine sentiments.'

In [90]:
#summary = []
#dataset = []

In [91]:
dataset.append("2015_06_negitive")

In [92]:
summary.append(response.choices[0].message.content)

In [93]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  6 non-null      object
 1   summary  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


In [94]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...


In [95]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5JQuBvjjNooLLzow0nSCUQ1Soi at 0x2435e598d10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various aspects of vaccination, particularly against the flu and other diseases. It highlights issues like low vaccination rates, painful experiences associated with getting vaccinated, and skepticism about vaccine efficacy, as well as concerns about potential health risks. Additionally, it mentions a resurgence of diseases like diphtheria due to anti-vaccine sentiments.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863445,
  "id": "chatcmpl-AUd5JQuBvjjNooLLzow0nSCUQ1Soi",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 66,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
  

## July

In [96]:
# Get monthly data
df_2015_07_neg = pd.read_csv('../data/local/df_neg_2015-07.csv').reset_index()
df_2015_07_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118 entries, 0 to 1117
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1118 non-null   int64  
 1   LocationCode            1118 non-null   object 
 2   LocationName            1118 non-null   object 
 3   StatesCode              1118 non-null   object 
 4   ...4                    1118 non-null   int64  
 5   id                      1118 non-null   float64
 6   text                    1118 non-null   object 
 7   label                   1118 non-null   object 
 8   in_reply_to_status_id   100 non-null    float64
 9   in_reply_to_user_id     111 non-null    float64
 10  quoted_user_id          10 non-null     float64
 11  quoted_status_id        10 non-null     float64
 12  retweeted_user_id       177 non-null    float64
 13  retweeted_status_id     177 non-null    float64
 14  created_at              1118 non-null   

### Representative sample based on week and location

In [97]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_07_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_07_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w26,Amapa,1,0.000894
1,2015-w26,Amazonas,3,0.002683
2,2015-w26,Bahia,2,0.001789
3,2015-w26,Ceara,2,0.001789
4,2015-w26,Distrito Federal,3,0.002683
...,...,...,...,...
103,2015-w31,Parana,1,0.000894
104,2015-w31,Pernambuco,2,0.001789
105,2015-w31,Rio Grande Do Sul,44,0.039356
106,2015-w31,Santa Catarina,2,0.001789


In [98]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w26,Amapa,1,0.000894,2
1,2015-w26,Amazonas,3,0.002683,6
2,2015-w26,Bahia,2,0.001789,4
3,2015-w26,Ceara,2,0.001789,4
4,2015-w26,Distrito Federal,3,0.002683,6
...,...,...,...,...,...
103,2015-w31,Parana,1,0.000894,2
104,2015-w31,Pernambuco,2,0.001789,4
105,2015-w31,Rio Grande Do Sul,44,0.039356,96
106,2015-w31,Santa Catarina,2,0.001789,4


In [99]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_07_neg_sample_size = pd.merge(df_2015_07_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [100]:
# Step 6: Perform the stratified sampling
#df_2015_07_neg_sample = df_2015_07_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_07_neg_sample = df_2015_07_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_07_neg_sample)} rows.")


Sampled 1118 rows.


In [101]:
text_gpt_2015_07_neg = '\n'.join(df_2015_07_neg_sample['text'].astype(str))

### Prompts and loops

In [102]:
prompt_7 = prompt_raw.replace('TWEETS', text_gpt_2015_07_neg)

In [103]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_7}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [104]:
response.choices[0].message.content

'The sentences discuss various opinions, experiences, and controversies surrounding vaccines. Many express discomfort and pain associated with vaccinations, while others mention skepticism, conspiracy theories, and criticisms of vaccination campaigns. There are references to specific vaccines (like HPV and anti-flu) and notable figures, including Jim Carrey, who criticize vaccination policies. Some highlight the lack of available vaccines, while others sarcastically question the efficacy of vaccines, suggesting they sometimes lead to illnesses instead of preventing them.'

In [105]:
#summary = []
#dataset = []

In [106]:
dataset.append("2015_07_negitive")

In [107]:
summary.append(response.choices[0].message.content)

In [108]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  7 non-null      object
 1   summary  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [109]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."


In [110]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5OZcLnI9PQJAmjVpPL71ypxD9B at 0x2435e72e860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss various opinions, experiences, and controversies surrounding vaccines. Many express discomfort and pain associated with vaccinations, while others mention skepticism, conspiracy theories, and criticisms of vaccination campaigns. There are references to specific vaccines (like HPV and anti-flu) and notable figures, including Jim Carrey, who criticize vaccination policies. Some highlight the lack of available vaccines, while others sarcastically question the efficacy of vaccines, suggesting they sometimes lead to illnesses instead of preventing them.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863450,
  "id": "chatcmpl-AUd5OZcLnI9PQJAmjVpPL71ypxD9B",
  "model": "gpt-4o-mini-2024-07-18",
  "obj

## August

In [111]:
# Get monthly data
df_2015_08_neg = pd.read_csv('../data/local/df_neg_2015-08.csv').reset_index()
df_2015_08_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207 entries, 0 to 1206
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1207 non-null   int64  
 1   LocationCode            1207 non-null   object 
 2   LocationName            1207 non-null   object 
 3   StatesCode              1207 non-null   object 
 4   ...4                    1207 non-null   int64  
 5   id                      1207 non-null   float64
 6   text                    1207 non-null   object 
 7   label                   1207 non-null   object 
 8   in_reply_to_status_id   113 non-null    float64
 9   in_reply_to_user_id     121 non-null    float64
 10  quoted_user_id          19 non-null     float64
 11  quoted_status_id        19 non-null     float64
 12  retweeted_user_id       195 non-null    float64
 13  retweeted_status_id     195 non-null    float64
 14  created_at              1207 non-null   

### Representative sample based on week and location

In [112]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_08_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_08_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w31,Alagoas,3,0.002486
1,2015-w31,Amazonas,5,0.004143
2,2015-w31,Bahia,6,0.004971
3,2015-w31,Ceara,1,0.000829
4,2015-w31,Distrito Federal,3,0.002486
...,...,...,...,...
88,2015-w35,Pernambuco,1,0.000829
89,2015-w35,Rio Grande Do Sul,19,0.015742
90,2015-w35,Santa Catarina,5,0.004143
91,2015-w35,Sao Paulo,30,0.024855


In [113]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w31,Alagoas,3,0.002486,6
1,2015-w31,Amazonas,5,0.004143,10
2,2015-w31,Bahia,6,0.004971,12
3,2015-w31,Ceara,1,0.000829,2
4,2015-w31,Distrito Federal,3,0.002486,6
...,...,...,...,...,...
88,2015-w35,Pernambuco,1,0.000829,2
89,2015-w35,Rio Grande Do Sul,19,0.015742,38
90,2015-w35,Santa Catarina,5,0.004143,10
91,2015-w35,Sao Paulo,30,0.024855,60


In [114]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_08_neg_sample_size = pd.merge(df_2015_08_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [115]:
# Step 6: Perform the stratified sampling
#df_2015_08_neg_sample = df_2015_08_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_08_neg_sample = df_2015_08_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_08_neg_sample)} rows.")


Sampled 1207 rows.


In [116]:
text_gpt_2015_08_neg = '\n'.join(df_2015_08_neg_sample['text'].astype(str))

### Prompts and loops

In [117]:
prompt_8 = prompt_raw.replace('TWEETS', text_gpt_2015_08_neg)

In [118]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_8}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [119]:
response.choices[0].message.content

'The sentences discuss various experiences and opinions related to vaccinations, expressing discomfort and pain from getting vaccinated, particularly in the arm or buttock. There are complaints about the soreness and side effects following vaccinations, as well as concerns about low vaccination rates among children. Some express disdain for vaccines, while others highlight the necessity of vaccinations for public health, including references to specific vaccines like those for polio and meningitis.'

In [120]:
#summary = []
#dataset = []

In [121]:
dataset.append("2015_08_negitive")

In [122]:
summary.append(response.choices[0].message.content)

In [123]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  8 non-null      object
 1   summary  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [124]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."
7,2015_08_negitive,The sentences discuss various experiences and ...


In [125]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5SahPOHXIKobr9FWQ2BtS9fNls at 0x2435e9f1f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss various experiences and opinions related to vaccinations, expressing discomfort and pain from getting vaccinated, particularly in the arm or buttock. There are complaints about the soreness and side effects following vaccinations, as well as concerns about low vaccination rates among children. Some express disdain for vaccines, while others highlight the necessity of vaccinations for public health, including references to specific vaccines like those for polio and meningitis.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863454,
  "id": "chatcmpl-AUd5SahPOHXIKobr9FWQ2BtS9fNls",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage

## September

In [126]:
# Get monthly data
df_2015_09_neg = pd.read_csv('../data/local/df_neg_2015-09.csv').reset_index()
df_2015_09_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   687 non-null    int64  
 1   LocationCode            687 non-null    object 
 2   LocationName            687 non-null    object 
 3   StatesCode              687 non-null    object 
 4   ...4                    687 non-null    int64  
 5   id                      687 non-null    float64
 6   text                    687 non-null    object 
 7   label                   687 non-null    object 
 8   in_reply_to_status_id   83 non-null     float64
 9   in_reply_to_user_id     85 non-null     float64
 10  quoted_user_id          7 non-null      float64
 11  quoted_status_id        7 non-null      float64
 12  retweeted_user_id       78 non-null     float64
 13  retweeted_status_id     78 non-null     float64
 14  created_at              687 non-null    ob

### Representative sample based on week and location

In [127]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_09_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_09_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w35,Alagoas,1,0.001456
1,2015-w35,Amapa,1,0.001456
2,2015-w35,Amazonas,2,0.002911
3,2015-w35,Bahia,1,0.001456
4,2015-w35,Ceara,1,0.001456
...,...,...,...,...
90,2015-w39,Rio Grande Do Sul,11,0.016012
91,2015-w39,Santa Catarina,3,0.004367
92,2015-w39,Sao Paulo,35,0.050946
93,2015-w39,Sergipe,2,0.002911


In [128]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w35,Alagoas,1,0.001456,3
1,2015-w35,Amapa,1,0.001456,3
2,2015-w35,Amazonas,2,0.002911,7
3,2015-w35,Bahia,1,0.001456,3
4,2015-w35,Ceara,1,0.001456,3
...,...,...,...,...,...
90,2015-w39,Rio Grande Do Sul,11,0.016012,39
91,2015-w39,Santa Catarina,3,0.004367,10
92,2015-w39,Sao Paulo,35,0.050946,124
93,2015-w39,Sergipe,2,0.002911,7


In [129]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_09_neg_sample_size = pd.merge(df_2015_09_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [130]:
# Step 6: Perform the stratified sampling
#df_2015_09_neg_sample = df_2015_09_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_09_neg_sample = df_2015_09_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_09_neg_sample)} rows.")


Sampled 687 rows.


In [131]:
text_gpt_2015_09_neg = '\n'.join(df_2015_09_neg_sample['text'].astype(str))

### Prompts and loops

In [132]:
prompt_9 = prompt_raw.replace('TWEETS', text_gpt_2015_09_neg)

In [133]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_9}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [134]:
response.choices[0].message.content

"The text discusses various individuals' negative sentiments and experiences regarding vaccinations, including fear, pain, and side effects. It highlights concerns about low vaccination coverage for diseases like polio and HPV in Brazil. Some express frustration over the perceived ineffectiveness or drawbacks of vaccines, while others mention logistical issues and the emotional difficulty of vaccinating children or pets. Overall, there is a strong emphasis on discomfort and skepticism toward vaccination."

In [135]:
#summary = []
#dataset = []

In [136]:
dataset.append("2015_09_negitive")

In [137]:
summary.append(response.choices[0].message.content)

In [138]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  9 non-null      object
 1   summary  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [139]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."
7,2015_08_negitive,The sentences discuss various experiences and ...
8,2015_09_negitive,The text discusses various individuals' negati...


In [140]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5WziEd1eC0Gkd6OlYsQ6YSYmva at 0x2435e814950> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various individuals' negative sentiments and experiences regarding vaccinations, including fear, pain, and side effects. It highlights concerns about low vaccination coverage for diseases like polio and HPV in Brazil. Some express frustration over the perceived ineffectiveness or drawbacks of vaccines, while others mention logistical issues and the emotional difficulty of vaccinating children or pets. Overall, there is a strong emphasis on discomfort and skepticism toward vaccination.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863458,
  "id": "chatcmpl-AUd5WziEd1eC0Gkd6OlYsQ6YSYmva",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  

## October

In [141]:
# Get monthly data
df_2015_10_neg = pd.read_csv('../data/local/df_neg_2015-10.csv').reset_index()
df_2015_10_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   750 non-null    int64  
 1   LocationCode            750 non-null    object 
 2   LocationName            750 non-null    object 
 3   StatesCode              750 non-null    object 
 4   ...4                    750 non-null    int64  
 5   id                      750 non-null    float64
 6   text                    750 non-null    object 
 7   label                   750 non-null    object 
 8   in_reply_to_status_id   61 non-null     float64
 9   in_reply_to_user_id     63 non-null     float64
 10  quoted_user_id          9 non-null      float64
 11  quoted_status_id        9 non-null      float64
 12  retweeted_user_id       138 non-null    float64
 13  retweeted_status_id     138 non-null    float64
 14  created_at              750 non-null    ob

### Representative sample based on week and location

In [142]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_10_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_10_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w40,Acre,1,0.001333
1,2015-w40,Amapa,2,0.002667
2,2015-w40,Amazonas,6,0.008000
3,2015-w40,Bahia,10,0.013333
4,2015-w40,Ceara,6,0.008000
...,...,...,...,...
84,2015-w44,Pernambuco,1,0.001333
85,2015-w44,Rio Grande Do Sul,8,0.010667
86,2015-w44,Santa Catarina,3,0.004000
87,2015-w44,Sao Paulo,16,0.021333


In [143]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w40,Acre,1,0.001333,3
1,2015-w40,Amapa,2,0.002667,6
2,2015-w40,Amazonas,6,0.008000,19
3,2015-w40,Bahia,10,0.013333,32
4,2015-w40,Ceara,6,0.008000,19
...,...,...,...,...,...
84,2015-w44,Pernambuco,1,0.001333,3
85,2015-w44,Rio Grande Do Sul,8,0.010667,26
86,2015-w44,Santa Catarina,3,0.004000,9
87,2015-w44,Sao Paulo,16,0.021333,52


In [144]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_10_neg_sample_size = pd.merge(df_2015_10_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [145]:
# Step 6: Perform the stratified sampling
#df_2015_10_neg_sample = df_2015_10_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_10_neg_sample = df_2015_10_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_10_neg_sample)} rows.")


Sampled 750 rows.


In [146]:
text_gpt_2015_10_neg = '\n'.join(df_2015_10_neg_sample['text'].astype(str))

### Prompts and loops

In [147]:
prompt_10 = prompt_raw.replace('TWEETS', text_gpt_2015_10_neg)

In [148]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_10}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [149]:
response.choices[0].message.content

'The text reflects widespread frustration and concern regarding vaccinations, highlighting individual experiences of pain and side effects after receiving vaccines. It discusses issues of neglect in vaccination programs, leading to the resurgence of diseases, and criticizes the inefficiency in accessing vaccines. There are also mentions of parental anxiety about their children being vaccinated and societal pressures surrounding vaccination.'

In [150]:
#summary = []
#dataset = []

In [151]:
dataset.append("2015_10_negitive")

In [152]:
summary.append(response.choices[0].message.content)

In [153]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  10 non-null     object
 1   summary  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [154]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."
7,2015_08_negitive,The sentences discuss various experiences and ...
8,2015_09_negitive,The text discusses various individuals' negati...
9,2015_10_negitive,The text reflects widespread frustration and c...


In [155]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5Z9lHhzSZUVy4tzm3LVayRg2fC at 0x2435e8c8400> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text reflects widespread frustration and concern regarding vaccinations, highlighting individual experiences of pain and side effects after receiving vaccines. It discusses issues of neglect in vaccination programs, leading to the resurgence of diseases, and criticizes the inefficiency in accessing vaccines. There are also mentions of parental anxiety about their children being vaccinated and societal pressures surrounding vaccination.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863461,
  "id": "chatcmpl-AUd5Z9lHhzSZUVy4tzm3LVayRg2fC",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 66,
    "completion_tokens_de

## November

In [156]:
# Get monthly data
df_2015_11_neg = pd.read_csv('../data/local/df_neg_2015-11.csv').reset_index()
df_2015_11_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   611 non-null    int64  
 1   LocationCode            611 non-null    object 
 2   LocationName            611 non-null    object 
 3   StatesCode              611 non-null    object 
 4   ...4                    611 non-null    int64  
 5   id                      611 non-null    float64
 6   text                    611 non-null    object 
 7   label                   611 non-null    object 
 8   in_reply_to_status_id   37 non-null     float64
 9   in_reply_to_user_id     42 non-null     float64
 10  quoted_user_id          7 non-null      float64
 11  quoted_status_id        7 non-null      float64
 12  retweeted_user_id       85 non-null     float64
 13  retweeted_status_id     85 non-null     float64
 14  created_at              611 non-null    ob

### Representative sample based on week and location

In [157]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_11_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_11_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w44,Bahia,3,0.004910
1,2015-w44,Ceara,4,0.006547
2,2015-w44,Distrito Federal,2,0.003273
3,2015-w44,Espirito Santo,2,0.003273
4,2015-w44,Goias,1,0.001637
...,...,...,...,...
86,2015-w48,Rio Grande Do Sul,5,0.008183
87,2015-w48,Roraima,1,0.001637
88,2015-w48,Santa Catarina,3,0.004910
89,2015-w48,Sao Paulo,22,0.036007


In [158]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w44,Bahia,3,0.004910,12
1,2015-w44,Ceara,4,0.006547,16
2,2015-w44,Distrito Federal,2,0.003273,8
3,2015-w44,Espirito Santo,2,0.003273,8
4,2015-w44,Goias,1,0.001637,4
...,...,...,...,...,...
86,2015-w48,Rio Grande Do Sul,5,0.008183,20
87,2015-w48,Roraima,1,0.001637,4
88,2015-w48,Santa Catarina,3,0.004910,12
89,2015-w48,Sao Paulo,22,0.036007,88


In [159]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_11_neg_sample_size = pd.merge(df_2015_11_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [160]:
# Step 6: Perform the stratified sampling
#df_2015_11_neg_sample = df_2015_11_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_11_neg_sample = df_2015_11_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_11_neg_sample)} rows.")


Sampled 611 rows.


In [161]:
text_gpt_2015_11_neg = '\n'.join(df_2015_11_neg_sample['text'].astype(str))

### Prompts and loops

In [162]:
prompt_11 = prompt_raw.replace('TWEETS', text_gpt_2015_11_neg)

In [163]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_11}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [164]:
response.choices[0].message.content

'The text discusses various experiences and feelings related to vaccinations, including pain, fear, and frustration. Many express discomfort after receiving vaccines, often experiencing soreness or adverse reactions. Additionally, there are concerns about vaccine shortages in Brazil and skepticism about their effectiveness. Some people reflect on the emotional toll of vaccinating pets and children. Overall, the sentiments range from annoyance to fear regarding vaccination processes and their outcomes.'

In [165]:
#summary = []
#dataset = []

In [166]:
dataset.append("2015_11_negitive")

In [167]:
summary.append(response.choices[0].message.content)

In [168]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  11 non-null     object
 1   summary  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [169]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."
7,2015_08_negitive,The sentences discuss various experiences and ...
8,2015_09_negitive,The text discusses various individuals' negati...
9,2015_10_negitive,The text reflects widespread frustration and c...


In [170]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5bcLdg8rBOqTXk3PSvLLYh1NzC at 0x2435ec07d10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various experiences and feelings related to vaccinations, including pain, fear, and frustration. Many express discomfort after receiving vaccines, often experiencing soreness or adverse reactions. Additionally, there are concerns about vaccine shortages in Brazil and skepticism about their effectiveness. Some people reflect on the emotional toll of vaccinating pets and children. Overall, the sentiments range from annoyance to fear regarding vaccination processes and their outcomes.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863463,
  "id": "chatcmpl-AUd5bcLdg8rBOqTXk3PSvLLYh1NzC",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "us

## December

In [171]:
# Get monthly data
df_2015_12_neg = pd.read_csv('../data/local/df_neg_2015-12.csv').reset_index()
df_2015_12_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1645 non-null   int64  
 1   LocationCode            1645 non-null   object 
 2   LocationName            1645 non-null   object 
 3   StatesCode              1645 non-null   object 
 4   ...4                    1645 non-null   int64  
 5   id                      1645 non-null   float64
 6   text                    1645 non-null   object 
 7   label                   1645 non-null   object 
 8   in_reply_to_status_id   93 non-null     float64
 9   in_reply_to_user_id     107 non-null    float64
 10  quoted_user_id          43 non-null     float64
 11  quoted_status_id        43 non-null     float64
 12  retweeted_user_id       445 non-null    float64
 13  retweeted_status_id     445 non-null    float64
 14  created_at              1645 non-null   

### Representative sample based on week and location

In [172]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2015_12_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2015_12_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2015-w48,Bahia,3,0.001824
1,2015-w48,Ceara,1,0.000608
2,2015-w48,Distrito Federal,3,0.001824
3,2015-w48,Espirito Santo,1,0.000608
4,2015-w48,Mato Grosso Do Sul,2,0.001216
...,...,...,...,...
101,2015-w53,Distrito Federal,1,0.000608
102,2015-w53,Mato Grosso Do Sul,1,0.000608
103,2015-w53,Paraiba,2,0.001216
104,2015-w53,Rio Grande Do Sul,3,0.001824


In [173]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2015-w48,Bahia,3,0.001824,4
1,2015-w48,Ceara,1,0.000608,1
2,2015-w48,Distrito Federal,3,0.001824,4
3,2015-w48,Espirito Santo,1,0.000608,1
4,2015-w48,Mato Grosso Do Sul,2,0.001216,2
...,...,...,...,...,...
101,2015-w53,Distrito Federal,1,0.000608,1
102,2015-w53,Mato Grosso Do Sul,1,0.000608,1
103,2015-w53,Paraiba,2,0.001216,2
104,2015-w53,Rio Grande Do Sul,3,0.001824,4


In [174]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2015_12_neg_sample_size = pd.merge(df_2015_12_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [175]:
# Step 6: Perform the stratified sampling
#df_2015_12_neg_sample = df_2015_12_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2015_12_neg_sample = df_2015_12_neg
# Step 7: Check the result
print(f"Sampled {len(df_2015_12_neg_sample)} rows.")


Sampled 1645 rows.


In [176]:
text_gpt_2015_12_neg = '\n'.join(df_2015_12_neg_sample['text'].astype(str))

### Prompts and loops

In [177]:
prompt_12 = prompt_raw.replace('TWEETS', text_gpt_2015_12_neg)

In [178]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_12}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [179]:
response.choices[0].message.content

'The text discusses controversies surrounding vaccines in Brazil, particularly the HPV vaccine and claims that a batch of expired rubella vaccine may have caused microcephaly in newborns. The MPF is petitioning to ban the HPV vaccine nationwide due to concerns over its effectiveness and safety. Many express distrust in government vaccination efforts, linking health issues to alleged vaccine failures or conspiracies.'

In [180]:
#summary = []
#dataset = []

In [181]:
dataset.append("2015_12_negitive")

In [182]:
summary.append(response.choices[0].message.content)

In [183]:
df_2015_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2015_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  12 non-null     object
 1   summary  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [184]:
df_2015_neg.tail(14)

,dataset,summary
0,2015_01_negitive,The text reflects a widespread frustration and...
1,2015_02_negitive,The selected texts express a mix of opinions a...
2,2015_03_negitive,The discussion revolves around various concern...
3,2015_04_negitive,The sentences express various opinions and exp...
4,2015_05_negitive,The text discusses various people's experience...
5,2015_06_negitive,The text discusses various aspects of vaccinat...
6,2015_07_negitive,"The sentences discuss various opinions, experi..."
7,2015_08_negitive,The sentences discuss various experiences and ...
8,2015_09_negitive,The text discusses various individuals' negati...
9,2015_10_negitive,The text reflects widespread frustration and c...


In [185]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd5fM5cfhzAHKCcL48oey1WTWAqI at 0x2435ec38a40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses controversies surrounding vaccines in Brazil, particularly the HPV vaccine and claims that a batch of expired rubella vaccine may have caused microcephaly in newborns. The MPF is petitioning to ban the HPV vaccine nationwide due to concerns over its effectiveness and safety. Many express distrust in government vaccination efforts, linking health issues to alleged vaccine failures or conspiracies.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863467,
  "id": "chatcmpl-AUd5fM5cfhzAHKCcL48oey1WTWAqI",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 74,
    "completion_tokens_details": {
      "accepted

## All 2015 negative

In [186]:
df_2015_neg.to_csv('../data/summary_2015_negative_tweets.csv')